In [1]:
import sys
sys.path.insert(0,'../../src')
sys.path.insert(0,'../../src/models')
ROOT_DIR = '../../Data/'
from utilities import * 
from logistic_utilities import *

C:\Users\nicol\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Prepare the inputs for the logistic regression

In [2]:
imgs, gt_imgs = LoadImages(0, root_dir = ROOT_DIR, verbose = 0)
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(imgs.shape[0])]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(gt_imgs.shape[0])]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

# Get X and Y
X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

# Cross Validation

In [3]:
degrees = np.array([2,3])
lambdas = np.array([100,1000])
best_lambda, best_degree, best_f1 = grid_search_hyperparam(Y,X,lambdas, degrees)


Grid search ====> 1/2 lambda starts...
Grid search ====> Lambda = 1.00e-02, degree = 2, F1-score = 0.515
Grid search ====> Lambda = 1.00e-02, degree = 3, F1-score = 0.520
Grid search ====> 2/2 lambda starts...
Grid search ====> Lambda = 1.00e-03, degree = 2, F1-score = 0.542
Grid search ====> Lambda = 1.00e-03, degree = 3, F1-score = 0.551

Grid search terminated.
Best lambda:  1000
Best degree:  3
Max F1-score:  0.5509541029261236


In [4]:
logreg = linear_model.LogisticRegression(C=best_lambda, class_weight="balanced")
poly = PolynomialFeatures(best_degree)
X = poly.fit_transform(X)
logreg.fit(X, Y)


LogisticRegression(C=1000, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [5]:
predict_and_submit_logistic(best_degree, logreg, 'Logistic.csv', root_dir = ROOT_DIR)

Loading test images...
Generating inputs from test images...
Predicting...
Submission generated in:  Logistic.csv
